In [1]:
from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
import pandas as pd

In [2]:
# Initialize APIs
nusc = NuScenes(version='v1.0-mini', dataroot='../data/sets/nuscenes', verbose=True)
nusc_can_bus = NuScenesCanBus(dataroot='../data/sets/nuscenes')

# Map scene_name to scene_token
scene_name_to_token = {scene['name']: scene['token'] for scene in nusc.scene}
scene_token_to_name = {v: k for k, v in scene_name_to_token.items()}

# Preload CAN messages per scene_name
can_by_scene = {
    scene_name: nusc_can_bus.get_messages(scene_name, 'vehicle_monitor')
    for scene_name in scene_name_to_token
}

data_rows = []

for sample in nusc.sample:
    sample_token = sample['token']
    sample_ts = sample['timestamp']
    scene_token = sample['scene_token']
    scene_name = scene_token_to_name[scene_token]

    can_msgs = can_by_scene.get(scene_name, [])

    # Find closest CAN message by timestamp
    if can_msgs:
        closest_can = min(can_msgs, key=lambda msg: abs(msg['utime'] - sample_ts))
    else:
        closest_can = None

    row = {
        'scene_name': scene_name,
        'scene_token': scene_token,
        'sample_token': sample_token,
        'timestamp': sample_ts
    }

    if closest_can:
        row.update({
            'can_timestamp': closest_can['utime'],
            'brake': closest_can['brake'],
            'brake_switch': closest_can['brake_switch'],
            'rear_left_rpm': closest_can['rear_left_rpm'],
            'rear_right_rpm' : closest_can['rear_right_rpm'],
            'can_utime': closest_can['utime'],
            'vehicle_speed': closest_can['vehicle_speed'],
            'yaw_rate': closest_can['yaw_rate']
        })

    data_rows.append(row)

# Convert to DataFrame
can_df = pd.DataFrame(data_rows)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.350 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
can_df.shape

(404, 12)

In [4]:
can_df.head()

,scene_name,scene_token,sample_token,timestamp,can_timestamp,brake,brake_switch,rear_left_rpm,rear_right_rpm,can_utime,vehicle_speed,yaw_rate
0,scene-0061,cc8c0bf57f984915a77078b10eb33198,ca9a282c9e77460f8360f564131a8af5,1532402927647951,1532402928127800,0,1,275.2617,276.1791,1532402928127800,31.44,0.6
1,scene-0061,cc8c0bf57f984915a77078b10eb33198,39586f9d59004284a7114a68825e8eec,1532402928147847,1532402928127800,0,1,275.2617,276.1791,1532402928127800,31.44,0.6
2,scene-0061,cc8c0bf57f984915a77078b10eb33198,356d81f38dd9473ba590f39e266f54e5,1532402928698048,1532402928627159,0,1,267.7557,267.7557,1532402928627159,30.73,0.6
3,scene-0061,cc8c0bf57f984915a77078b10eb33198,e0845f5322254dafadbbed75aaa07969,1532402929197353,1532402929127810,0,2,257.2473,256.5384,1532402929127810,29.45,0.5
4,scene-0061,cc8c0bf57f984915a77078b10eb33198,c923fe08b2ff4e27975d2bf30934383b,1532402929697797,1532402929627057,0,1,244.4037,246.7389,1532402929627057,28.09,0.2


In [5]:
can_df.to_excel('CanInfo.xlsx', index=False)